<a href="https://colab.research.google.com/github/profteachkids/CHE3023/blob/main/NRTL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import jax.numpy as jnp
import jax
R=8.314

In [9]:
# For the 3 component mixture: (1) Methanol, (2) Ethanol, and (3) Water system, 
# the matrices A, C, D are zeros, while B and alpha have been determined experimentally.
B=jnp.array( [[   0.,   -155.5,   -24.49],
 [ 190.06,    0.,    -55.17],
 [ 307.17,  670.44,    0.  ]])

alpha = jnp.array( [[0.,    0.305, 0.3  ],
 [0.305, 0.,    0.303],
 [0.3,   0.303, 0.   ]])

In [10]:
x12 = jnp.array([0.2,0.3])

In [11]:
def Gex(x,T):
  tau = B/T
  G = jnp.exp(-alpha * tau)
  return jnp.dot(x, jnp.matmul(x,tau*G)/jnp.matmul(x,G))

In [12]:
jnp.exp(jax.grad(Gex,0)(jnp.array([0.2,0.3,0.5]),300))

DeviceArray([1.0061771, 1.335594 , 1.4216974], dtype=float32)

In [13]:
def lngamma(x12,T):
    x = jnp.append(x12,1- x12.sum())
    tau = B/T
    G = jnp.exp(-alpha * tau)
    xG=jnp.matmul(x,G)
    xtauGdivxG = jnp.matmul(x,tau * G) / xG
    print(tau, xtauGdivxG)
    lngamma = xtauGdivxG+jnp.matmul(G * (tau - xtauGdivxG), x/xG)
    return lngamma

In [14]:
lngamma(x12,300)

[[ 0.         -0.5183333  -0.08163333]
 [ 0.6335333   0.         -0.1839    ]
 [ 1.0239      2.2348      0.        ]] [ 0.6542138   0.5661521  -0.07343629]


DeviceArray([0.00615811, 0.2893762 , 0.3518515 ], dtype=float32)

In [15]:

tfm = jnp.eye(2) + jnp.broadcast_to(x12,(2,2))*jax.jacobian(lngamma,0)(x12,300)[:2,:]

[[ 0.         -0.5183333  -0.08163333]
 [ 0.6335333   0.         -0.1839    ]
 [ 1.0239      2.2348      0.        ]] Traced<ConcreteArray([ 0.6542138   0.5661521  -0.07343629], dtype=float32)>with<JVPTrace(level=2/0)> with
  primal = DeviceArray([ 0.6542138 ,  0.5661521 , -0.07343629], dtype=float32)
  tangent = Traced<ShapedArray(float32[3])>with<JaxprTrace(level=1/0)> with
    pval = (ShapedArray(float32[3]), *)
    recipe = JaxprEqnRecipe(eqn_id=<object object at 0x7f7a2555c640>, in_tracers=(Traced<ConcreteArray([0.8150511  0.78828764 1.0221496 ], dtype=float32):JaxprTrace(level=1/0)>, Traced<ConcreteArray([ 0.53321767  0.4462907  -0.07506287], dtype=float32):JaxprTrace(level=1/0)>, Traced<ConcreteArray([1.5053253  1.6092762  0.95713043], dtype=float32):JaxprTrace(level=1/0)>, Traced<ShapedArray(float32[3]):JaxprTrace(level=1/0)>, Traced<ShapedArray(float32[3]):JaxprTrace(level=1/0)>), out_tracer_refs=[<weakref at 0x7f7a116d9830; to 'JaxprTracer' at 0x7f7a117400b0>], out_avals=[Sha

In [16]:
tfm

DeviceArray([[ 1.0089608 , -0.19754106],
             [-0.22314115,  0.5773254 ]], dtype=float32)

In [17]:
dxdz = jnp.array([0.3,0.4])
d = jnp.matmul(tfm,dxdz)
d

DeviceArray([0.22367185, 0.16398782], dtype=float32)